In [2]:
from entities import *
from constants import TEAMS_DATA, STAT_EFFECTS

import time
import requests
from bs4 import BeautifulSoup, Comment

import dowhy
from tqdm import tqdm, trange

import os
import pickle
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import warnings

# Suppress specific pandas FutureWarnings regarding Series.__getitem__
warnings.filterwarnings("ignore", category=FutureWarning, message=".*Series.__getitem__ treating keys as positions is deprecated.*")
warnings.filterwarnings("ignore", category=FutureWarning, message=".*The default of observed=False is deprecated.*")

c:\Users\amgal\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
with open(TEAMS_DATA, 'rb') as f:
    teams = pickle.load(f)

Getting a collection of 5 players based on how often they play:

In [4]:
dfs = []

for team in tqdm(teams.keys()):
    dfs.append(teams[team].causal_df)

df = pd.concat(dfs, ignore_index=True)
df.dropna(inplace=True)
df.head()

100%|██████████| 110/110 [00:00<?, ?it/s]


,Team,Opp_Name,Tm,Opp_Score,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,oFG,oFGA,oFG%,o3P,o3PA,o3P%,o2P,o2PA,o2P%,oeFG%,oFT,oFTA,oFT%,oORB,oDRB,oTRB,oAST,oSTL,oBLK,oTOV,oPF
0,Abilene Christian,McMurry,92.0,55.0,34.0,69.0,0.493,6.0,20.0,0.300,28.0,49.0,0.571,0.536,18.0,26.0,0.692,17.0,34.0,51.0,21.0,7.0,7.0,13.0,15.0,18.0,63.0,0.286,8.0,31.0,0.258,10.0,32.0,0.313,0.349,11.0,15.0,0.733,7.0,18.0,25.0,10.0,6.0,5.0,16.0,20.0
1,Abilene Christian,Omaha,73.0,71.0,25.0,57.0,0.439,7.0,21.0,0.333,18.0,36.0,0.500,0.500,16.0,21.0,0.762,9.0,18.0,27.0,15.0,8.0,3.0,14.0,23.0,21.0,45.0,0.467,6.0,19.0,0.316,15.0,26.0,0.577,0.533,23.0,31.0,0.742,3.0,16.0,19.0,10.0,11.0,3.0,15.0,20.0
2,Abilene Christian,SW Adventist,104.0,63.0,43.0,68.0,0.632,9.0,22.0,0.409,34.0,46.0,0.739,0.699,9.0,12.0,0.750,11.0,23.0,34.0,26.0,9.0,3.0,11.0,16.0,22.0,50.0,0.440,4.0,15.0,0.267,18.0,35.0,0.514,0.480,15.0,18.0,0.833,4.0,11.0,15.0,6.0,7.0,0.0,19.0,12.0
3,Abilene Christian,Stephen F. Austin,66.0,76.0,24.0,57.0,0.421,2.0,10.0,0.200,22.0,47.0,0.468,0.439,16.0,19.0,0.842,8.0,27.0,35.0,12.0,4.0,5.0,10.0,21.0,24.0,53.0,0.453,13.0,23.0,0.565,11.0,30.0,0.367,0.575,15.0,26.0,0.577,4.0,22.0,26.0,14.0,5.0,5.0,7.0,15.0
4,Abilene Christian,Texas State,49.0,63.0,17.0,55.0,0.309,5.0,15.0,0.333,12.0,40.0,0.300,0.355,10.0,19.0,0.526,12.0,15.0,27.0,6.0,14.0,5.0,14.0,19.0,21.0,46.0,0.457,5.0,8.0,0.625,16.0,38.0,0.421,0.511,16.0,22.0,0.727,10.0,23.0,33.0,8.0,11.0,9.0,18.0,15.0


## Building our DAG

In [5]:
graph = """
digraph{

Tm;
STL -> oTOV;
STL -> PF -> oFTA -> oFT -> Opp_Score;

BLK -> o2P;
BLK -> o3P;
BLK -> oPF;

TOV -> 3PA -> 3P -> Tm;
TOV -> 2PA -> 2P -> Tm;
TOV -> o3PA -> o3P -> Opp_Score;
TOV -> o2PA -> o2P -> Opp_Score;

ORB -> 3PA;
ORB -> 2PA;

DRB -> o2PA;
DRB -> o3PA;
DRB -> TOV;

Opp_Score;
oSTL -> TOV;
oSTL -> oPF -> FTA -> FT -> Tm;

oBLK -> 2P;
oBLK -> 3P;
oBLK -> PF;

oTOV -> 3PA;
oTOV -> 2PA;
oTOV -> o3PA;
oTOV -> o2PA;

oORB -> o3PA;
oORB -> o2PA;

oDRB -> 2PA;
oDRB -> 3PA;
oDRB -> oTOV;
}
"""

In [6]:
model = dowhy.CausalModel(data=df, graph=graph, treatment="2P",outcome="Tm")

<img src="resources\visualizations\graphviz_graph.png" width="500" />

## Estimating Average Causal Effects

In [7]:
OUTCOMES = ['Tm', 'Opp_Score']

VARIABLES = ['2P',  '2PA',  '3P',  '3PA',  'FT',  'FTA',  'STL',  'BLK',  'TOV',  'PF',  'ORB',  'DRB',  # Team Stats
            'o2P', 'o2PA', 'o3P', 'o3PA', 'oFT', 'oFTA', 'oSTL', 'oBLK', 'oTOV', 'oPF', 'oORB', 'oDRB']  # Opponent Stats

In [8]:
treatment_effects = {}
treatment_refute = {}
# for treatment in tqdm(VARIABLES):
    
#     outcome_effects = {}
#     outcome_refute = {}
#     for outcome in OUTCOMES:

#         # Creating the Causal Model
#         model = dowhy.CausalModel(
#             data=df,
#             graph=graph,
#             treatment=treatment,
#             outcome=outcome
#         )

#         # Identifying the Estimand
#         identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)

#         # Estimating the Estimand
#         estimate = model.estimate_effect(identified_estimand,
#                                  method_name="backdoor.linear_regression",
#                                  target_units="ate")
        
#         outcome_effects[outcome] = float(estimate.value)
#         outcome_refute[outcome] = model.refute_estimate(identified_estimand, estimate, method_name="placebo_treatment_refuter", show_progress_bar=True, placebo_type="permute")
#     treatment_effects[treatment] = outcome_effects
#     treatment_refute[treatment] = outcome_refute
        

for treatment in tqdm(VARIABLES):
    outcome_effects = {}
    outcome_refute = {}
    for outcome in OUTCOMES:
        
        model = dowhy.CausalModel(
            data=df,
            graph=graph,
            treatment=treatment,
            outcome=outcome
        )

        identified_estimand = model.identify_effect(proceed_when_unidentifiable=False)

        estimate = model.estimate_effect(
            identified_estimand,
            method_name="backdoor.linear_regression",
            target_units="ate"
        )
        
        outcome_effects[outcome] = float(estimate.value)

        if not identified_estimand.estimands:
            outcome_refute[outcome] = 'Skipped, due to lack of estimand.'
            continue

        refutation = model.refute_estimate(
            identified_estimand, 
            estimate, 
            method_name="placebo_treatment_refuter", 
            show_progress_bar=False,
            placebo_type="permute"
        )
        outcome_refute[outcome] = refutation
        
    treatment_effects[treatment] = outcome_effects
    treatment_refute[treatment] = outcome_refute

    with open(STAT_EFFECTS, 'wb') as f:
        pickle.dump([treatment_effects, treatment_refute], f)

100%|██████████| 24/24 [25:47:03<00:00, 3867.66s/it]   


In [9]:
treatment_effects

{'2P': {'Tm': 2.017006811689555, 'Opp_Score': 0.0},
 '2PA': {'Tm': 1.1496965234754697, 'Opp_Score': 0.0},
 '3P': {'Tm': 2.9717002538341504, 'Opp_Score': 0.0},
 '3PA': {'Tm': 0.8873538911671659, 'Opp_Score': 0.0},
 'FT': {'Tm': 0.9818173102965133, 'Opp_Score': 0.0},
 'FTA': {'Tm': 0.7575435968587172, 'Opp_Score': 0.0},
 'STL': {'Tm': 0.8589943530838013, 'Opp_Score': -0.7534334965279896},
 'BLK': {'Tm': 0.1378401297097298, 'Opp_Score': -0.6418120036156125},
 'TOV': {'Tm': -0.8425473739779221, 'Opp_Score': 0.7406190508702082},
 'PF': {'Tm': 0.0, 'Opp_Score': 1.096555548790839},
 'ORB': {'Tm': 0.21598461015534554, 'Opp_Score': 0.0},
 'DRB': {'Tm': 0.7581812974921291, 'Opp_Score': -0.856969143678171},
 'o2P': {'Tm': 0.0, 'Opp_Score': 2.0466993773063678},
 'o2PA': {'Tm': 0.0, 'Opp_Score': 1.0394173675516143},
 'o3P': {'Tm': 0.0, 'Opp_Score': 2.998410625322869},
 'o3PA': {'Tm': 0.0, 'Opp_Score': 0.9543668853428429},
 'oFT': {'Tm': 0.0, 'Opp_Score': 1.006618404689597},
 'oFTA': {'Tm': 0.0, 'Op

# Refuting the Estimates

In [11]:
with open(STAT_EFFECTS, 'rb') as f:
    treatment_effects = pickle.load(f)

In [12]:
treatment_effects[0]

{'2P': {'Tm': 2.017006811689555, 'Opp_Score': 0.0},
 '2PA': {'Tm': 1.1496965234754697, 'Opp_Score': 0.0},
 '3P': {'Tm': 2.9717002538341504, 'Opp_Score': 0.0},
 '3PA': {'Tm': 0.8873538911671659, 'Opp_Score': 0.0},
 'FT': {'Tm': 0.9818173102965133, 'Opp_Score': 0.0},
 'FTA': {'Tm': 0.7575435968587172, 'Opp_Score': 0.0},
 'STL': {'Tm': 0.8589943530838013, 'Opp_Score': -0.7534334965279896},
 'BLK': {'Tm': 0.1378401297097298, 'Opp_Score': -0.6418120036156125},
 'TOV': {'Tm': -0.8425473739779221, 'Opp_Score': 0.7406190508702082},
 'PF': {'Tm': 0.0, 'Opp_Score': 1.096555548790839},
 'ORB': {'Tm': 0.21598461015534554, 'Opp_Score': 0.0},
 'DRB': {'Tm': 0.7581812974921291, 'Opp_Score': -0.856969143678171},
 'o2P': {'Tm': 0.0, 'Opp_Score': 2.0466993773063678},
 'o2PA': {'Tm': 0.0, 'Opp_Score': 1.0394173675516143},
 'o3P': {'Tm': 0.0, 'Opp_Score': 2.998410625322869},
 'o3PA': {'Tm': 0.0, 'Opp_Score': 0.9543668853428429},
 'oFT': {'Tm': 0.0, 'Opp_Score': 1.006618404689597},
 'oFTA': {'Tm': 0.0, 'Op

['2P',
 '2PA',
 '3P',
 '3PA',
 'FT',
 'FTA',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'ORB',
 'DRB',
 'o2P',
 'o2PA',
 'o3P',
 'o3PA',
 'oFT',
 'oFTA',
 'oSTL',
 'oBLK',
 'oTOV',
 'oPF',
 'oORB',
 'oDRB']

In [31]:
for key, val in list(treatment_effects[1].items()):
    print(f"""
=========================================================
{key}
- Team Score
    {val['Tm']}
- Opponent Score
    {val['Opp_Score']}
""")


2P
- Team Score
    Refute: Use a Placebo Treatment
Estimated effect:2.017006811689555
New effect:-0.0035639585933093087
p value:0.92

- Opponent Score
    Skipped, due to lack of estimand.


2PA
- Team Score
    Refute: Use a Placebo Treatment
Estimated effect:1.1496965234754697
New effect:0.0018508142305513787
p value:0.8400000000000001

- Opponent Score
    Skipped, due to lack of estimand.


3P
- Team Score
    Refute: Use a Placebo Treatment
Estimated effect:2.9717002538341504
New effect:-0.002531897941168353
p value:0.88

- Opponent Score
    Skipped, due to lack of estimand.


3PA
- Team Score
    Refute: Use a Placebo Treatment
Estimated effect:0.8873538911671659
New effect:0.000709502415633665
p value:0.9199999999999999

- Opponent Score
    Skipped, due to lack of estimand.


FT
- Team Score
    Refute: Use a Placebo Treatment
Estimated effect:0.9818173102965133
New effect:0.0007116552153266297
p value:0.96

- Opponent Score
    Skipped, due to lack of estimand.


FTA
- Team